# Analyse Isolated Star Sources

 work with Weakly_2024_07
- use jupyter kernel LSST


- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab
- creation date : 2024/02/28
- update : 2024/03/01

``Quantum Graphs``: https://tigress-web.princeton.edu/~lkelvin/pipelines/ 


In [ ]:
#pip install ipympl

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from ipywidgets import widgets
#%matplotlib widget
#%matplotlib ipympl
    
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm,SymLogNorm


import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)
from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch
from astropy.io import fits
import astropy.coordinates as coord
import astropy.units as u
import pandas as pd

from astropy.time import Time
from datetime import datetime, timedelta

import matplotlib.ticker                         # here's where the formatter is
import os
import re
import pandas as pd

plt.rcParams["figure.figsize"] = (4,3)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'

In [ ]:
#filesource_in="preSourceTable_202312.csv"
filesource_in="../data/202402/isolated_star_sources_202402.csv"

In [ ]:
df = pd.read_csv(filesource_in,index_col=0)
df.dropna(axis=1, how='all',inplace =True)
df = df.rename(columns={"visit": "visitId"})
df["nightObs"] =df.apply(lambda x: x['visitId']//100_000, axis=1)
df["seq"] = df["visitId"] - df["nightObs"]*100_000
col_df = df.columns.tolist()
df.sort_values(by="visitId",inplace=True)

In [ ]:
df

In [ ]:
col_df

In [ ]:
def convert_fluxtomag(x) :
    """
    The object and source catalogs store only fluxes. There are hundreds of flux-related columns, 
    and to store them also as magnitudes would be redundant, and a waste of space.
    All flux units are nanojanskys. The AB Magnitudes Wikipedia page provides a concise resource 
    for users unfamiliar with AB magnitudes and jansky fluxes. To convert to AB magnitudes use:
    As demonstrated in Section 2.3.2, to add columns of magnitudes after retrieving columns of flux, users can do this:
    results_table['r_calibMag'] = -2.50 * numpy.log10(results_table['r_calibFlux']) + 31.4
    results_table['r_cModelMag'] = -2.50 * numpy.log10(results_table['r_cModelFlux']) + 31.4
    (from DP0 tutorial)
    """
    return -2.50 * np.log10(x) + 31.4

In [ ]:
df["apFlux_35_0_instMag"] =  df['apFlux_35_0_instFlux'].map(convert_fluxtomag)
df["apFlux_50_0_instMag"] =  df['apFlux_50_0_instFlux'].map(convert_fluxtomag)

In [ ]:
column_names = ["apFlux_35_0_instMag","apFlux_50_0_instMag"]
fig,ax = plt.subplots(figsize=(14,8))
axes = df.hist(column= column_names,by="band",bins=20,ax=ax,xrot=0,range=(16,23),legend=True)
plt.suptitle(column_names)

In [ ]:
np.sort(df["nightObs"].unique())

# Statistics per nights

In [ ]:
df_stat = df.groupby(by=["nightObs","band","tract"]).count()["index"]

In [ ]:
df_stat

In [ ]:
type(df_stat)

In [ ]:
df_stat_pertract = df.groupby(by=["nightObs","tract"]).count()["index"]

In [ ]:
df_stat_pertract

# Handle pandas series with composite index
https://stackoverflow.com/questions/25386870/pandas-plotting-with-multi-index

In [ ]:
ax = df_stat_pertract.unstack(level=1).plot(kind='bar', subplots=False, rot=90,figsize=(16,8),grid=True)
plt.tight_layout()

In [ ]:
ax = df_stat_pertract.unstack(level=1).plot(kind='bar', subplots=True, rot=90,figsize=(16,30),grid=True)
plt.tight_layout()

## Select the tracts

In [ ]:
TRACT_SEL = 3864

In [ ]:
df_stat = df[df["tract"] == TRACT_SEL ].groupby(by=["nightObs","band"]).count()["index"]

In [ ]:
df_stat

In [ ]:
df_stat.unstack(level=1)[["g","r","i","z","y"]]

In [ ]:
ax = df_stat.unstack(level=1)[["g","r","i","z","y"]].plot(kind='bar', subplots=False, rot=90,figsize=(16,8),grid=True,color=["g","r","orange","grey","k"])
plt.tight_layout()
plt.title(f"Number of visits in tract {TRACT_SEL}")
plt.ylabel("number of visits per night")

In [ ]:
df_stat_unstack = df_stat.unstack(level=1)[["g","r","i","z","y"]]

In [ ]:
def convertNumToDatestr(num):
    year = num//10_000
    month= (num-year*10_000)//100
    day = (num-year*10_000-month*100)

    year_str = str(year).zfill(4)
    month_str = str(month).zfill(2)
    day_str = str(day).zfill(2)
    
    datestr = f"{year_str}-{month_str}-{day_str}"
    return pd.to_datetime(datestr)

In [ ]:
all_datetimes = [convertNumToDatestr(num) for num in df_stat_unstack.index ]
df_stat_unstack["datetimes"] = all_datetimes
df_stat_unstack = df_stat_unstack.set_index("datetimes")

In [ ]:
df_stat_unstack.index

In [ ]:
import matplotlib.dates as mdates
month_locator = mdates.MonthLocator(interval=1)
day_locator = mdates.DayLocator(interval=1)
year_month_formatter = mdates.DateFormatter("%Y-%m") 
fig, ax = plt.subplots(1,1,figsize=(16, 10))



# fit 1
#df_stat_unstack.plot( kind='bar', rot=0,ax=ax)
df_stat_unstack.plot( marker="+",markersize=30,lw=0,color=["g","r","orange","grey","k"],ax=ax,legend=False)
df_stat_unstack.plot( marker="o",markersize=10,lw=0,color=["g","r","orange","grey","k"],ax=ax)
#df_stat_unstack.plot( kind="bar",x=df_stat_unstack.index,color=["g","r","orange","grey","k"],ax=ax)

#ax.set_xticks(ax.get_xticks(), ax.get_xticklabels(), rotation=45, ha='right')
#ax.set_xticklabels(labels=df_stat_unstack.index, rotation=70, rotation_mode="anchor", ha="right");
ax.xaxis.set_major_locator(month_locator) # Locator for major axis only.
ax.xaxis.set_minor_locator(day_locator)
ax.xaxis.set_major_formatter(year_month_formatter)
ax.grid()
#plt.tight_layout()
ax.set_ylabel("number of visits per night")
ax.set_title(f"Number of visits in tract {TRACT_SEL}")

In [ ]:
import matplotlib.dates as mdates
month_locator = mdates.MonthLocator(interval=1)
day_locator = mdates.DayLocator(interval=1)
year_month_formatter = mdates.DateFormatter("%Y-%m") 
dt = timedelta(hours=12)
fig, ax = plt.subplots(1,1,figsize=(16, 6))

ax.bar(df_stat_unstack.index-2*dt,df_stat_unstack["g"],color="g",label="g")
ax.bar(df_stat_unstack.index-dt,df_stat_unstack["r"],color="r",label="r")
ax.bar(df_stat_unstack.index+dt,df_stat_unstack["i"],color="orange",label="i")
ax.bar(df_stat_unstack.index+2*dt,df_stat_unstack["z"],color="grey",label="z")
ax.bar(df_stat_unstack.index+3*dt,df_stat_unstack["y"],color="k",label="y")
ax.legend()

ax.xaxis.set_major_locator(month_locator) # Locator for major axis only.
ax.xaxis.set_minor_locator(day_locator)
ax.xaxis.set_major_formatter(year_month_formatter)
ax.set_xticks(ax.get_xticks(), ax.get_xticklabels(), rotation=45, ha='right')
ax.grid()
#plt.tight_layout()
ax.set_xlabel("date")
ax.set_ylabel("number of visits per night")
ax.set_title(f"Number of visits in tract {TRACT_SEL}")

## Select a date

In [ ]:
DATEOBS =  20240130
cut = (df["nightObs"] == DATEOBS)
df = df[cut]

In [ ]:
all_visits = df["visitId"].unique()

In [ ]:
all_visits

In [ ]:
cmap = matplotlib.cm.inferno
cmap_reversed = matplotlib.cm.get_cmap('inferno_r')

In [ ]:
MAGCUT = 17.8
cuts = [ (df["band"] == "g") & (df["apFlux_50_0_instMag"] < MAGCUT), 
         (df["band"] == "r") & (df["apFlux_50_0_instMag"] < MAGCUT),
         (df["band"] == "z") & (df["apFlux_50_0_instMag"] < MAGCUT),
         (df["band"] == "y") & (df["apFlux_50_0_instMag"] < MAGCUT) ]
all_bands = ["g","r","z","y"]
NB = len(all_bands)
all_df_cuts_inbands = [ df[cuts[idx]] for idx in range(NB)]
for visit in all_visits:
    fig, axs = plt.subplots(ncols=4,nrows=1,figsize=(16,4))
    for ib in range(NB):     
        my_colors = all_df_cuts_inbands[ib]["apFlux_50_0_instMag"].values
        if len(my_colors) > 0:
            all_df_cuts_inbands[ib].plot.scatter('x','y',c=my_colors,colormap='jet_r',grid=True,ax = axs[ib])
            axs[ib].set_title(all_bands[ib])
            axs[ib].axes.set_aspect('equal')
            axs[ib].axes.set_xlim(0.,4000.)
            axs[ib].axes.set_ylim(0.,4000.)
    #axs[0].scatter(data=all_df_cuts_inbands[0], x='x', y='y', c='apFlux_35_0_instMag', cmap="viridis")
    title = f"Isolated star pixel coordinates : visit {visit} , tract = {TRACT_SEL}"
    plt.suptitle(title)
    plt.tight_layout()

In [ ]:
for visit in all_visits:
    fig, axs = plt.subplots(ncols=4,nrows=1,figsize=(16,4))
    for ib in range(NB):     
        my_colors = all_df_cuts_inbands[ib]["apFlux_50_0_instMag"].values
        if len(my_colors) > 0:
            all_df_cuts_inbands[ib].plot.scatter('ra','dec',c=my_colors,colormap='jet_r',grid=True,ax = axs[ib])
            axs[ib].set_title(all_bands[ib])
            axs[ib].axes.set_aspect('equal')
            
    #axs[0].scatter(data=all_df_cuts_inbands[0], x='x', y='y', c='apFlux_35_0_instMag', cmap="viridis")
    title = f"Isolated star radec coordinates : visit {visit} , tract = {TRACT_SEL}"
    plt.suptitle(title)
    
    plt.tight_layout()